# Satellite Imagery for ARI Manual Classification Workflow <img align="right" src="../../Supplementary_data/dea_logo.jpg">

* **Compatability:** Notebook currently compatible with the `NCI` and `DEA Sandbox` environments
* **Products used:** 
[s2a_ard_granule](https://explorer.sandbox.dea.ga.gov.au/s2a_ard_granule), 
[s2b_ard_granule](https://explorer.sandbox.dea.ga.gov.au/s2b_ard_granule),
[ga_ls5t_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3),
[ga_ls7e_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls7e_ard_3),
[ga_ls8c_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls8c_ard_3)

## Background
The ability to extract pixel boundaries as a shapefile or other vector file type is very useful for remote sensing applications where the boundaries of the pixel are needed. 
This is useful for matching drone imagery with remotely sensed imagery. The Fractional Cover of Water project requires us to use satellite imagery from Landsat and Sentinel 2 satellites that match our drone imagery from our field sites. We take the matching imagery, find the pixel edges, and then manually classify what is in the pixels (what type of wet vegetation we have and what fraction of the pixel it covers), in order to create input data for the algorithm. 


## Description
The notebook uses drone imagery to classify the land cover type, including `Floating, Emergent, OpenWater, GreenVeg(PV), DryVeg(NPV), BareSoil(BS), Forel-Ule water colour (if the type is OpenWater)`

* first bring in the drone data in resolution (1, -1) (unit meter)
* classify the pixels into categories listed above
* save the results as raster into `geotiff`

To use the notebook, please refer the instruction video and doc linked below
- xxx
- xxx
***

## Get started

### 1. Upload drone image (refer the instruction doc), then find out the file name by

In [ ]:
!ls

### 2. Set the variable `drone_tif_path` accordingly, 
e.g. `drone_tif_path = "./Sbends_Orthomosaic_export_TueAug18040306687120.tif"` from above

In [ ]:
# change the file name if necessary
drone_tif_path = "./Sbends_Orthomosaic_export_TueAug18040306687120.tif"

### 3. `Run -> Run All Cells`

In [ ]:
import sys
import os
from os import path
import datacube
import rasterio.features
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import shape
import xarray as xr
import re
from datetime import datetime
import urllib

from datacube.utils.cog import write_cog
from datacube.utils.geometry import assign_crs
from datacube.utils.geometry import GeoBox
from odc.algo import xr_reproject

from rasterio import warp

from bokeh.io import curdoc, output_notebook, show
from bokeh.layouts import layout, column, row
from bokeh.models import (CheckboxGroup, Select, ColumnDataSource, HoverTool, YearsTicker, Legend,
                          CustomJS, LegendItem, field, Range1d, Circle, Button, RadioGroup, TextInput, WheelZoomTool,
                          ResetTool, BoxZoomTool, SaveTool, LinearColorMapper, CategoricalColorMapper, 
                          Label, PreText, FileInput, Toggle)
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models.glyphs import Text
from bokeh.palettes import Blues256
from bokeh.colors import RGB, named
from bokeh.plotting import figure

In [ ]:
from Scripts.dea_dask import create_local_dask_cluster

In [ ]:
output_notebook()

In [ ]:
create_local_dask_cluster()

## Load the drone orthomosaic for your wetland of interest
The path here points to the example_data folder; change if necessary.

In [ ]:
# fill the local path of output raster or can be changed later
results_tif_path = './test.tif'
# don't change this unless required
drone_res_tgt = (1, 1)

In [ ]:
def load_drone_tif(fname, res_tgt):
    drone = xr.open_rasterio(fname, parse_coordinates=True, chunks={'band': 1, 'x': 1024, 'y': 1024})
    drone = assign_crs(drone)
    affine, width, height = warp.calculate_default_transform(drone.crs, 'EPSG:3577', drone.shape[1], drone.shape[2],
                                             *drone.geobox.extent.boundingbox)
    tgt_affine, tgt_width, tgt_height = warp.aligned_target(affine, width, height, res_tgt)
    drone_geobox = GeoBox(tgt_width, tgt_height, tgt_affine, 'EPSG:3577')
    drone_tgt = xr_reproject(drone, drone_geobox, resampling= 'bilinear' )
    return drone_tgt.load()

def load_results_tif(fname):
    results = xr.open_rasterio(fname, parse_coordinates=True, chunks={'band': 1, 'x': 1024, 'y': 1024})
    return results.load()

In [ ]:
drone_tgt = load_drone_tif(drone_tif_path, drone_res_tgt)
rgba_image = np.empty((drone_tgt.shape[1], drone_tgt.shape[2]), dtype='uint32')
view = rgba_image.view(dtype='uint8').reshape(drone_tgt.shape[1], drone_tgt.shape[2], drone_tgt.shape[0])
for i in range(3):
    view[:, :, i] = (drone_tgt.data[i]).astype('uint8')
view[:, :, 3] = 255
scale_factor = np.array([drone_tgt.x.data.min() if drone_tgt.x.data.min() > 0 else drone_tgt.x.data.max(),
                         drone_tgt.y.data.min() if drone_tgt.y.data.min() > 0 else drone_tgt.y.data.max()])
xr_results = xr.DataArray(data=[np.zeros(rgba_image.shape, dtype='uint8')] * 2, dims=['band', 'y', 'x'], 
                     coords={'band': [1, 2], 
                     'y': drone_tgt.coords['y'], 'x':drone_tgt.coords['x']}, attrs={'nodata':0})

In [ ]:
def plot_doc(doc):
    drone_source = ColumnDataSource(data={'img': [np.flip(rgba_image, axis=0)]})
    results_source = ColumnDataSource(data={'category': [np.zeros(rgba_image.shape, dtype='uint8')],
                                           'forel_ule': [np.zeros(rgba_image.shape, dtype='uint8')]})
    
    drone_file_input = TextInput(value=drone_tif_path, title="Load drone imagery", height=50, width=600,
                                sizing_mode='fixed')
    def open_drone_tif(attrname, old, new):
        fname = drone_file_input.value
        if not path.exists(fname):
            print("file doesn't exist!")
            return
        drone_tgt = load_drone_tif(fname, drone_res_tgt)
        rgba_image = np.empty((drone_tgt.shape[1], drone_tgt.shape[2]), dtype='uint32')
        view = rgba_image.view(dtype='uint8').reshape(drone_tgt.shape[1], drone_tgt.shape[2], drone_tgt.shape[0])
        for i in range(3):
            view[:, :, i] = (drone_tgt.data[i]).astype('uint8')
        view[:, :, 3] = 255
        scale_factor = np.array([drone_tgt.x.data.min() if drone_tgt.x.data.min() > 0 else drone_tgt.x.data.max(),
                         drone_tgt.y.data.min() if drone_tgt.y.data.min() > 0 else drone_tgt.y.data.max()])
        drone_source.data['img'] = [np.flip(rgba_image, axis=0)]
        
    drone_file_input.on_change('value', open_drone_tif)
    
    result_file_input = TextInput(value="./test.tif", title="Results tiff file", height=50, width=600,
                                sizing_mode='fixed')
    result_save_button = Button(label="Save results", button_type="success")
    result_load_button = Button(label="Load results", button_type="success")
    def open_result_tif(event):
        fname = result_file_input.value
        if not path.exists(fname):
            print("file doesn't exist!")
            return
        global xr_results
        xr_results = load_results_tif(fname)
        results_source.data['category'] = [np.flip(xr_results[0].data, axis=0)]
        results_source.data['forel_ule'] = [np.flip(xr_results[1].data, axis=0)]
        
    def save_result_tif(event):
        fname = result_file_input.value
        if path.exists(fname):
            time_now = str(datetime.now()).replace(':', '').replace(' ', '').replace('-', '')
            os.rename(fname, fname + '.bak' + time_now)
        write_cog(xr_results, fname)
        
    result_load_button.on_click(open_result_tif)
    result_save_button.on_click(save_result_tif)
    
    drone_fig = figure(tooltips=[('x-cood', "$x{0.0}"), ('y-coord', "$y{0.0}")], title="image %s" %("drone"), 
            x_axis_type='auto', y_axis_type='auto', x_minor_ticks=10, y_minor_ticks=10,
            x_axis_label="x origin %s" % scale_factor[0],
            y_axis_label="y origin %s" % scale_factor[1])
    drone_fig.toolbar.active_scroll = drone_fig.select_one(WheelZoomTool)
    drone_tag = ['drone', 1]
    drone_fig.image_rgba(image='img', source=drone_source, 
                x=drone_tgt.x.data.min()-scale_factor[0],
                y=drone_tgt.y.data.min()-scale_factor[1],
               dw=drone_tgt.shape[2], dh=drone_tgt.shape[1],
                         tags=drone_tag,
                         level="image")
    transparent_white = RGB(255, 255, 255, 0)
    cats_color = [named.lawngreen.to_rgb(), named.limegreen.to_rgb(), named.olive.to_rgb(),
                  named.deepskyblue.to_rgb(), named.darkgreen.to_rgb(), named.beige.to_rgb(),
                  named.brown.to_rgb()]
    cats_color_mapper = LinearColorMapper(cats_color, low=1, high=len(cats_color), low_color=transparent_white)
    water_color_mapper = LinearColorMapper(Blues256, low=1, high=21, low_color=transparent_white)
    water_tag = ['forel_ule', 21]
    cats_tag = ['cats', 10]
    water_image = drone_fig.image(image='forel_ule', source=results_source, x=drone_tgt.x.data.min()-scale_factor[0],
                y=drone_tgt.y.data.min()-scale_factor[1],
                dw=drone_tgt.shape[2], dh=drone_tgt.shape[1], 
                color_mapper=water_color_mapper,
                global_alpha=0.8,
                level="image", tags=water_tag)
    
    cats_image = drone_fig.image(image='category', source=results_source, x=drone_tgt.x.data.min()-scale_factor[0],
                y=drone_tgt.y.data.min()-scale_factor[1],
                dw=drone_tgt.shape[2], dh=drone_tgt.shape[1], 
                color_mapper=cats_color_mapper,
                global_alpha=0.8,
                level="image", tags=cats_tag)
   
    coords_label = PreText(text="null", width=100, sizing_mode='fixed')
    js_code = """
        var ind_x = Math.floor(cb_obj.x);
        var ind_y = Math.floor(cb_obj.y);
        var data_s = source.data;
        console.log("x:", ind_x);
        console.log("y:", ind_y);
        target.text = "x=" + ind_x.toString() +";" + "y=" + ind_y.toString();
    """
    js_callback = CustomJS(args={'source': drone_source, 'target': coords_label}, 
                           code=js_code)
    drone_fig.js_on_event('tap', js_callback)
    
    def get_ind_from_coords():
        if (coords_label.text == "null"):
            print("pick a pixel please")
            return (None, None)
        coords = coords_label.text.split(';')
        ind_x = coords[0].split('=')[1]
        ind_y = coords[1].split('=')[1]
        return (abs(int(ind_y)), abs(int(ind_x)))

    cats = ["NA", "Overstory","Emergent","Floating","OpenWater","GreenVeg","DryVeg","Bare"]  
    radio_group = RadioGroup(labels=cats, active=0, height=800, height_policy="fixed", aspect_ratio=0.1)
    forel_ule_scale = TextInput(value="0", title="Forel-Ule Water Colour", width=100, sizing_mode='fixed')  
    def choose_cat(attrname, old, new):
        ind_y, ind_x = get_ind_from_coords()
        if ind_y is None or ind_x is None:
            return
        if attrname == 'active':
            xr_results[0][ind_y, ind_x] = radio_group.active
            results_source.data['category'] = [np.flip(xr_results[0].data, axis=0)]
        if attrname == 'value':  
            check_numbers = re.match(r'^[0-9]+$', forel_ule_scale.value)
            if check_numbers is None:
                print("only input numbers", forel_ule_scale.value)
                forel_ule_scale.value = "0"
            elif int(forel_ule_scale.value) < 0 or int(forel_ule_scale.value) > 21:
                print("invalid value, please check!")
                forel_ule_scale.value = "0"
            elif radio_group.active != 4 and int(forel_ule_scale.value) > 0:
                forel_ule_scale.value = "0"
                print("cannot set value for non-water")
            xr_results[1][ind_y, ind_x] = int(forel_ule_scale.value)
            results_source.data['forel_ule'] = [np.flip(xr_results[1].data, axis=0)]
            
    radio_group.on_change('active', choose_cat)
    forel_ule_scale.on_change('value', choose_cat)
    
    def coords_change(attrname, old, new):
        ind_y, ind_x = get_ind_from_coords()
        if ind_y is None or ind_x is None:
            return
        radio_group.active = xr_results[0].data[ind_y, ind_x]
        forel_ule_scale.value = str(xr_results[1].data[ind_y, ind_x])
        
    coords_label.on_change('text', coords_change)
                  
    overlay_toggle_category = Toggle(label="Overlay category", button_type="success",
                                     height=50, width=150, sizing_mode='fixed', active=True)
    overlay_toggle_water_color = Toggle(label="Overlay water color", button_type="success", 
                                        height=50, width=150, sizing_mode='fixed', active=True)
    
    def overlay_results(event):
        if (overlay_toggle_water_color.active):
            water_image.visible = True
        else:
            water_image.visible = False
        if (overlay_toggle_category.active):
            cats_image.visible = True
        else:
            cats_image.visible = False
            
    overlay_toggle_category.on_click(overlay_results)
    overlay_toggle_water_color.on_click(overlay_results)
    
    control_group = column(coords_label, forel_ule_scale, radio_group)
    result_group = row(result_file_input, column(result_load_button, result_save_button))
    layouts = layout([drone_file_input, result_group, [overlay_toggle_category, overlay_toggle_water_color],
                      [control_group, drone_fig]], sizing_mode='scale_height')
    doc.add_root(layouts)

In [ ]:
def remote_jupyter_proxy_url(port):
    """
    Callable to configure Bokeh's show method when a proxy must be
    configured.

    If port is None we're asking about the URL
    for the origin header.
    """
    base_url = "https://app.sandbox.dea.ga.gov.au/"
    host = urllib.parse.urlparse(base_url).netloc
    # If port is None we're asking for the URL origin
    # so return the public hostname.
    if port is None:
        return host

    service_url_path = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    proxy_url_path = 'proxy/%d' % port

    user_url = urllib.parse.urljoin(base_url, service_url_path)
    full_url = urllib.parse.urljoin(user_url, proxy_url_path)
    return full_url

In [ ]:
# if you know your url
# notebook_url = "http://localhost:8889"
show(plot_doc, notebook_url=remote_jupyter_proxy_url)

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** August 2020

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)